**bold text**# Unit 2 Assignment: Mixture of Experts (MoE) Router
### Smart Customer Support Router using Groq API

**Architecture:**
- **Router** → Classifies the user query
- **Orchestrator** → Picks the right expert
- **Experts** → Technical / Billing / General / (Bonus: Tool-Use)

## Step 1: Install Required Libraries

In [1]:
!pip install groq python-dotenv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.8 MB/s eta 0:00:00


## Step 2: Set Up API Key

> 💡 In Colab, we use `getpass` so your key is never shown on screen.

In [9]:
import os
from getpass import getpass

# Enter your Groq API Key securely
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API Key: ")

print("API Key set successfully!")

Enter your Groq API Key: ··········
API Key set successfully!


## Step 3: Define the Experts (MODEL_CONFIG)

Each expert has a **system prompt** that gives it a specific personality and focus area.
We use the same base model (`mixtral-8x7b-32768`) but with different system prompts — this is the core idea of MoE simulation.

In [23]:
from groq import Groq

# Initialize the Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])

# Base model to use for all experts
BASE_MODEL = "mixtral-8x7b-32768"

# -------------------------------------------------------
# MODEL_CONFIG: Dictionary of all our expert configurations
# -------------------------------------------------------
MODEL_CONFIG = {
    "technical": {
        "name": "Technical Expert ",
        "system_prompt": (
            "You are a highly skilled Technical Support Engineer with deep expertise in "
            "software development, debugging, APIs, and system architecture. "
            "You communicate in a precise, rigorous, and code-focused manner. "
            "Always provide code snippets where relevant. "
            "Explain errors clearly, identify root causes, and suggest best-practice fixes."
        ),
        "temperature": 0.7
    },
    "billing": {
        "name": "Billing Expert ",
        "system_prompt": (
            "You are a compassionate and professional Billing Support Specialist. "
            "You handle refund requests, subscription issues, and payment disputes with empathy. "
            "Always acknowledge the customer's concern first, then explain the policy clearly. "
            "Guide users through the steps to resolve their billing issue. "
            "Be polite, patient, and solution-oriented."
        ),
        "temperature": 0.7
    },
    "general": {
        "name": "General Assistant ",
        "system_prompt": (
            "You are a friendly and helpful general-purpose customer support assistant. "
            "Answer questions in a warm, conversational tone. "
            "If you're unsure about something, politely say so and offer to help the user find the right department."
        ),
        "temperature": 0.7
    },
    # BONUS: Tool-Use Expert (mock data fetcher)
    "tool_use": {
        "name": "Tool-Use Expert ",
        "system_prompt": (
            "You are a data assistant that fetches and explains real-time information. "
            "Present the data clearly and explain what it means for the user."
        ),
        "temperature": 0.5
    }
}

print("MODEL_CONFIG defined with experts:", list(MODEL_CONFIG.keys()))

MODEL_CONFIG defined with experts: ['technical', 'billing', 'general', 'tool_use']


In [25]:
# Fix: Update to a supported model
BASE_MODEL = "llama-3.3-70b-versatile"  # Current recommended replacement

print("Model updated to:", BASE_MODEL)

Model updated to: llama-3.3-70b-versatile


## Step 4: The Router

The router uses the LLM with `temperature=0` (for **consistency**) to classify the user's query into one of: `technical`, `billing`, `general`, or `tool_use` (bonus).

In [26]:
def route_prompt(user_input: str) -> str:
    """
    Routes the user query to the correct expert category.

    Args:
        user_input (str): The raw query from the user.

    Returns:
        str: One of ['technical', 'billing', 'general', 'tool_use']
    """

    # Router system prompt: very strict, forces single-word output
    router_system_prompt = """
    You are a query classification router. Your ONLY job is to classify the user's message.

    Classify into EXACTLY ONE of these categories:
    - technical   → bugs, code errors, API issues, software problems, programming
    - billing     → payments, charges, refunds, subscriptions, invoices, pricing
    - tool_use    → requests for real-time data like crypto prices, weather, stock prices
    - general     → everything else (greetings, general questions, other topics)

    Rules:
    - Return ONLY the single category word. Nothing else.
    - No punctuation, no explanation, no extra words.
    - Example output: technical
    """

    response = client.chat.completions.create(
        model=BASE_MODEL,
        messages=[
            {"role": "system", "content": router_system_prompt},
            {"role": "user", "content": user_input}
        ],
        temperature=0  # temperature=0 for deterministic, consistent routing
    )

    # Extract and clean the category
    category = response.choices[0].message.content.strip().lower()

    # Safety fallback: if LLM returns something unexpected, default to 'general'
    valid_categories = list(MODEL_CONFIG.keys())
    if category not in valid_categories:
        print(f"   Router returned unknown category '{category}', defaulting to 'general'")
        category = "general"

    return category


# Quick test of the router
print(" Testing the Router:")
print("  'IndexError in my code' ->", route_prompt("I'm getting an IndexError in my code"))
print("  'Charged twice this month' ->", route_prompt("I was charged twice this month"))
print("  'Hello, how are you?' ->", route_prompt("Hello, how are you?"))
print("  'Bitcoin price now' ->", route_prompt("What's the current price of Bitcoin?"))

 Testing the Router:
  'IndexError in my code' -> technical
  'Charged twice this month' -> billing
  'Hello, how are you?' -> general
  'Bitcoin price now' -> tool_use


## Step 5: Bonus - Mock Tool-Use Function

This simulates fetching real-time data (like a crypto price API).

In [27]:
import random

# Mock database of prices (simulating a real API)
MOCK_PRICES = {
    "bitcoin": 67423.50,
    "ethereum": 3512.80,
    "dogecoin": 0.142,
    "solana": 172.35
}

def mock_fetch_price(user_input: str) -> str:
    """
    Mock tool that simulates fetching cryptocurrency prices.
    In a real app, this would call an actual crypto price API.
    """
    user_input_lower = user_input.lower()

    for coin, price in MOCK_PRICES.items():
        if coin in user_input_lower:
            # Add small random variation to simulate live prices
            live_price = price + random.uniform(-100, 100)
            return f" [MOCK TOOL RESULT] Current {coin.capitalize()} price: ${live_price:,.2f} USD (simulated live data)"

    return " [MOCK TOOL RESULT] Sorry, I couldn't find price data for that asset. Try: Bitcoin, Ethereum, Dogecoin, or Solana."


print(" Testing Mock Tool:")
print(mock_fetch_price("What is the price of Bitcoin?"))
print(mock_fetch_price("How much is Ethereum?"))

 Testing Mock Tool:
 [MOCK TOOL RESULT] Current Bitcoin price: $67,355.42 USD (simulated live data)
 [MOCK TOOL RESULT] Current Ethereum price: $3,490.25 USD (simulated live data)


## Step 6: The Orchestrator

The `process_request()` function ties everything together:
1. Calls the **router** to classify the query
2. Picks the right **expert** config
3. Calls the **LLM** with that expert's system prompt (or uses the tool for `tool_use`)
4. Returns the final answer

In [28]:
def process_request(user_input: str) -> str:
    """
    Main orchestrator function for the MoE Router.

    Args:
        user_input (str): The user's query.

    Returns:
        str: The expert's response.
    """

    print(f"\n{'='*60}")
    print(f"User Query: {user_input}")
    print(f"{'='*60}")

    # --- STEP 1: Route the query ---
    print(" Step 1: Routing query...")
    category = route_prompt(user_input)
    expert = MODEL_CONFIG[category]
    print(f" Routed to: {expert['name']} (category: '{category}')")

    # --- STEP 2: Handle tool_use separately (Bonus) ---
    if category == "tool_use":
        print("🔧 Step 2: Calling Mock Tool...")
        tool_result = mock_fetch_price(user_input)

        # Pass tool result to the LLM to generate a natural response
        print(" Step 3: Expert generating response from tool data...")
        response = client.chat.completions.create(
            model=BASE_MODEL,
            messages=[
                {"role": "system", "content": expert["system_prompt"]},
                {"role": "user", "content": f"{user_input}\n\nTool Data: {tool_result}"}
            ],
            temperature=expert["temperature"]
        )
        final_answer = response.choices[0].message.content

    # --- STEP 2 (normal): Use the expert's system prompt ---
    else:
        print(" Step 2: Expert generating response...")
        response = client.chat.completions.create(
            model=BASE_MODEL,
            messages=[
                {"role": "system", "content": expert["system_prompt"]},
                {"role": "user", "content": user_input}
            ],
            temperature=expert["temperature"]
        )
        final_answer = response.choices[0].message.content

    # --- STEP 3: Return the final answer ---
    print(f"\n {expert['name']} Response:")
    print("-" * 60)
    print(final_answer)
    print("-" * 60)

    return final_answer


print(" Orchestrator function defined!")

 Orchestrator function defined!


## Step 7: Run the Full Demo

Let's test all 4 expert paths with example queries from the assignment.

In [29]:
#  Test 1: Technical Query
process_request("My Python script is throwing an IndexError on line 5. How do I fix it?")


User Query: My Python script is throwing an IndexError on line 5. How do I fix it?
 Step 1: Routing query...
 Routed to: Technical Expert  (category: 'technical')
 Step 2: Expert generating response...

 Technical Expert  Response:
------------------------------------------------------------
# Step-by-step analysis of the problem:
1. **IndexError** is typically raised when you're trying to access an element in a list (or other sequence type) that doesn't exist. For example, if you have a list `my_list = [1, 2, 3]`, trying to access `my_list[3]` would raise an **IndexError** because list indices in Python are 0-based, meaning they start at 0 and end at `len(my_list) - 1`.
2. Without the actual code, it's hard to pinpoint the exact issue. However, we can discuss a common scenario that leads to this error and how to fix it.
3. A common cause of **IndexError** is iterating over a list and trying to access an index that is out of range. This can happen if you're manually incrementing an in

"# Step-by-step analysis of the problem:\n1. **IndexError** is typically raised when you're trying to access an element in a list (or other sequence type) that doesn't exist. For example, if you have a list `my_list = [1, 2, 3]`, trying to access `my_list[3]` would raise an **IndexError** because list indices in Python are 0-based, meaning they start at 0 and end at `len(my_list) - 1`.\n2. Without the actual code, it's hard to pinpoint the exact issue. However, we can discuss a common scenario that leads to this error and how to fix it.\n3. A common cause of **IndexError** is iterating over a list and trying to access an index that is out of range. This can happen if you're manually incrementing an index variable without properly checking its bounds.\n\n# Fixed solution:\nLet's assume your code looks something like this:\n```python\nmy_list = [1, 2, 3]\nfor i in range(5):  # This will cause an IndexError\n    print(my_list[i])\n```\nTo fix this, you should ensure that your loop doesn't

In [30]:
#  Test 2: Billing Query
process_request("I was charged twice for my subscription this month. I want a refund.")


User Query: I was charged twice for my subscription this month. I want a refund.
 Step 1: Routing query...
 Routed to: Billing Expert  (category: 'billing')
 Step 2: Expert generating response...

 Billing Expert  Response:
------------------------------------------------------------
I'm so sorry to hear that you were charged twice for your subscription this month. I can imagine how frustrating that must be for you. I'm here to help and want to assure you that we'll do our best to resolve this issue as quickly as possible.

To initiate the refund process, I'll need to verify some information with you. Could you please provide me with your subscription details, such as your account name and the date of the duplicate charge? This will help me to look into the matter further.

Our policy is to refund any unauthorized or duplicate charges within 3-5 business days. I'll make sure to process the refund for the incorrect charge as soon as possible. If you've already paid for the current mont

"I'm so sorry to hear that you were charged twice for your subscription this month. I can imagine how frustrating that must be for you. I'm here to help and want to assure you that we'll do our best to resolve this issue as quickly as possible.\n\nTo initiate the refund process, I'll need to verify some information with you. Could you please provide me with your subscription details, such as your account name and the date of the duplicate charge? This will help me to look into the matter further.\n\nOur policy is to refund any unauthorized or duplicate charges within 3-5 business days. I'll make sure to process the refund for the incorrect charge as soon as possible. If you've already paid for the current month's subscription, the refund will be applied to the duplicate charge.\n\nTo expedite the process, I'll also need you to confirm a few details. Please let me know if you have received an email confirmation for both charges, and if you have any screenshots or transaction IDs related

In [31]:
#  Test 3: General Query
process_request("Hi! I'm new here. What services do you offer?")


User Query: Hi! I'm new here. What services do you offer?
 Step 1: Routing query...
 Routed to: General Assistant  (category: 'general')
 Step 2: Expert generating response...

 General Assistant  Response:
------------------------------------------------------------
Welcome to our support team. I'm so glad you're here. We offer a wide range of services to help with pretty much anything you might need. From answering questions about our products and services, to troubleshooting issues, to just generally helping you find what you're looking for - we're here to help.

We can assist with everything from billing and account inquiries, to technical support and troubleshooting, to even just providing information about our company and what we do. If you have a specific question or issue in mind, I'd be happy to try and help you out. Or, if you're just looking for some general information, I can definitely point you in the right direction.

What's on your mind today? Is there something specif

"Welcome to our support team. I'm so glad you're here. We offer a wide range of services to help with pretty much anything you might need. From answering questions about our products and services, to troubleshooting issues, to just generally helping you find what you're looking for - we're here to help.\n\nWe can assist with everything from billing and account inquiries, to technical support and troubleshooting, to even just providing information about our company and what we do. If you have a specific question or issue in mind, I'd be happy to try and help you out. Or, if you're just looking for some general information, I can definitely point you in the right direction.\n\nWhat's on your mind today? Is there something specific you're looking for help with, or do you just want to get a sense of what we're all about?"

In [32]:
#  Test 4: Bonus - Tool Use Query
process_request("What is the current price of Bitcoin?")


User Query: What is the current price of Bitcoin?
 Step 1: Routing query...
 Routed to: Tool-Use Expert  (category: 'tool_use')
🔧 Step 2: Calling Mock Tool...
 Step 3: Expert generating response from tool data...

 Tool-Use Expert  Response:
------------------------------------------------------------
The current price of Bitcoin is $67,513.02 USD. This is a real-time value, which means it's subject to constant fluctuations based on market activity. The price of Bitcoin can change rapidly due to various factors such as market demand, global events, and economic conditions.

To put this into perspective, the current price of $67,513.02 USD indicates the amount of money you would need to buy one Bitcoin at this exact moment. If you're considering buying or selling Bitcoin, it's essential to keep an eye on the price, as it can change significantly in a short period.

Please note that this price is simulated live data and may not reflect the actual current price of Bitcoin. For the most a

"The current price of Bitcoin is $67,513.02 USD. This is a real-time value, which means it's subject to constant fluctuations based on market activity. The price of Bitcoin can change rapidly due to various factors such as market demand, global events, and economic conditions.\n\nTo put this into perspective, the current price of $67,513.02 USD indicates the amount of money you would need to buy one Bitcoin at this exact moment. If you're considering buying or selling Bitcoin, it's essential to keep an eye on the price, as it can change significantly in a short period.\n\nPlease note that this price is simulated live data and may not reflect the actual current price of Bitcoin. For the most accurate and up-to-date information, I recommend checking a reliable cryptocurrency exchange or financial website."

## Step 8: Interactive Mode

Try your own queries below!

In [33]:
#  Type your own query here and run the cell!
my_query = "My API is returning a 401 Unauthorized error. How do I debug this?"

process_request(my_query)


User Query: My API is returning a 401 Unauthorized error. How do I debug this?
 Step 1: Routing query...
 Routed to: Technical Expert  (category: 'technical')
 Step 2: Expert generating response...

 Technical Expert  Response:
------------------------------------------------------------
# Debugging 401 Unauthorized Error in API

## Introduction

A 401 Unauthorized error indicates that the request sent to the API was not authenticated or the authentication credentials were invalid. To debug this issue, we need to investigate the authentication mechanism used by the API and verify that the credentials are being sent correctly.

## Step 1: Verify Authentication Mechanism
Check the API documentation to determine the authentication mechanism used (e.g., Basic Auth, Bearer Token, OAuth). Ensure that you understand the required authentication headers or parameters.

## Step 2: Check Request Headers
Inspect the request headers to ensure that the authentication credentials are being sent corr

'# Debugging 401 Unauthorized Error in API\n\n## Introduction\n\nA 401 Unauthorized error indicates that the request sent to the API was not authenticated or the authentication credentials were invalid. To debug this issue, we need to investigate the authentication mechanism used by the API and verify that the credentials are being sent correctly.\n\n## Step 1: Verify Authentication Mechanism\nCheck the API documentation to determine the authentication mechanism used (e.g., Basic Auth, Bearer Token, OAuth). Ensure that you understand the required authentication headers or parameters.\n\n## Step 2: Check Request Headers\nInspect the request headers to ensure that the authentication credentials are being sent correctly. For example, if using Bearer Token authentication, verify that the `Authorization` header is present with the correct token:\n```http\nGET /api/endpoint HTTP/1.1\nAuthorization: Bearer YOUR_API_TOKEN\n```\nIn code, you can set the `Authorization` header using the followin

---

##  Summary: How the MoE Router Works

```
User Query
    │
    ▼
┌─────────────────────────────────┐
│         ROUTER (LLM)            │
│  temperature=0 (consistent)     │
│  "Classify into: technical /    │
│   billing / general / tool_use" │
└─────────────────────────────────┘
    │
    ▼  Returns category name
┌─────────────────────────────────┐
│        ORCHESTRATOR             │
│  Picks expert from MODEL_CONFIG │
└─────────────────────────────────┘
    │
    ├──► technical → Technical Expert System Prompt → LLM
    ├──► billing   → Billing Expert System Prompt   → LLM
    ├──► general   → General Expert System Prompt   → LLM
    └──► tool_use  → Mock Tool → Tool Result        → LLM
                                                        │
                                               Final Answer
```

**Key Concepts Used:**
- **MoE Architecture**: Same base model, different system prompts = different experts
- **Router**: LLM with `temperature=0` for deterministic classification
- **Orchestrator**: Logic layer that selects and calls the right expert
- **Tool Use (Bonus)**: Mock function simulating real-time data fetching